<a href="https://colab.research.google.com/github/jenny-bogart/grocery-basket-analysis/blob/main/notebooks/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning Code
Takes Grocery Store Dataset (https://www.kaggle.com/datasets/irfanasrullah/groceries) and standardizes it.
Final Output:

1. Item(s): # of items in basket
2. Item 1: First item purchased in transaction
3. Item 2: Second item purchased in transaction (if applicable)
...
4. Item 32: Thirty second item purchased in transaction (if applicable)





In [13]:
!pip install kaggle

In [67]:
import pandas as pd
import re
from IPython.utils.text import string
from datetime import date
import pickle

In [1]:
# Upload kaggle JSON key
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"jennybogart21","key":"0ead098ec5ad53fe4741d484b21f071d"}'}

In [2]:
# Download Kaggle dataset
# URL: https://www.kaggle.com/datasets/irfanasrullah/groceries
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d irfanasrullah/groceries

Dataset URL: https://www.kaggle.com/datasets/irfanasrullah/groceries
License(s): copyright-authors
groceries.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# Read grocery store dataset to a Dataframe
!unzip groceries.zip
import pandas as pd
df = pd.read_csv("groceries - groceries.csv")
df.head()

Archive:  groceries.zip
replace groceries - groceries.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: groceries - groceries.csv  
replace groceries.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: groceries.csv           


,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Drop Dulicates
df.drop_duplicates(inplace=True)

In [43]:
# Ensure Item(s) is numeric
df['Item(s)'] = pd.to_numeric(df['Item(s)'], errors='coerce')

# Define item columns
item_cols = [f'Item {i}' for i in range(1, 33)]

# Clean item columns
df[item_cols] = (
    df[item_cols]
    .fillna('')
    .astype(str)
    .apply(lambda col: col.str.lower().str.strip())
)

In [44]:
# create a list for each transaction
transactions = df[item_cols].apply(
    lambda row: [item for item in row if item != ''],
    axis=1
)
# remove duplicates
transactions = transactions.apply(lambda x: list(set(x)))

# remove empty transactions
transactions = transactions[transactions.apply(len) > 0]



After Data Cleaning, Complete Sanity Checks:
1. Are all values in transactions strings?
2. Check basket sizes (are they reasonable?)
3. How many unique items are there?
4. Look at sample data


In [45]:
# are there any numbers in the final output?
any(item.isnumeric() for t in transactions for item in t)

False

In [46]:
# What is the average basket size? - This makes sense, there are no empty transactions (min length > 1) and
# the max transaction length (31) is a reasonable number
print(transactions.apply(len).describe())

count    7011.000000
mean        5.649265
std         3.548252
min         1.000000
25%         3.000000
50%         5.000000
75%         7.000000
max        32.000000
dtype: float64


In [48]:
# Compare reported item count vs actual non-empty items
calculated_counts = df[item_cols].apply(
    lambda row: sum(item != '' for item in row),
    axis=1
)

df['count_matches'] = df['Item(s)'] == calculated_counts
# prints any values that do not have the same number of Item(s) as they do in transactions
df[df['count_matches'] == False]

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32,count_matches,count_diff,count


In [63]:
# checks if number of items is equal in original df and transactions list
# finds unique items in original df (excluding empty strings)
og_df_unique_vals = pd.unique(df[item_cols].values.ravel())
og_df_unique_vals = [v for v in og_df_unique_vals if pd.notna(v) and v != '']
print(len(og_df_unique_vals))
# finds the unique items in transactions df
transaction_unique_vals = set(item for t in transactions for item in t)
len(transaction_unique_vals)

169


169

In [71]:
transactions_df = transactions.to_frame()
transactions_df.rename(columns={'0': 'transactions'}, inplace=True)

In [72]:
# Save to CSV
df.to_csv("grocery_transactions_clean.csv", index=False)
# save as Pandas series (use for next python scripts)
transactions.to_pickle("transactions_clean.pkl")